In [1]:
import pandas as pd
import yfinance as yf

In [5]:
amd = yf.Ticker('amd')

In [6]:
amd.info

{'previousClose': 51.71,
 'regularMarketOpen': 52.2803,
 'twoHundredDayAverage': 35.94151,
 'trailingAnnualDividendYield': None,
 'payoutRatio': None,
 'volume24Hr': None,
 'regularMarketDayHigh': 52.81,
 'navPrice': None,
 'averageDailyVolume10Day': 52387557,
 'totalAssets': None,
 'regularMarketPreviousClose': 51.71,
 'fiftyDayAverage': 46.11697,
 'trailingAnnualDividendRate': None,
 'open': 52.2803,
 'averageVolume10days': 52387557,
 'expireDate': None,
 'yield': None,
 'algorithm': None,
 'dividendRate': None,
 'exDividendDate': None,
 'beta': None,
 'circulatingSupply': None,
 'startDate': None,
 'regularMarketDayLow': 49.47,
 'priceHint': 2,
 'currency': 'USD',
 'trailingPE': 263.61255,
 'regularMarketVolume': 80004909,
 'lastMarket': None,
 'maxSupply': None,
 'openInterest': None,
 'marketCap': 58761969664,
 'volumeAllCurrencies': None,
 'strikePrice': None,
 'averageVolume': 54508625,
 'priceToSalesTrailing12Months': None,
 'dayLow': 49.47,
 'ask': 50.27,
 'ytdReturn': None,
 

In [8]:
amd_history = amd.history(period='max')

In [9]:
amd_df = pd.DataFrame(amd_history)
amd_df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1980-03-17,0.0,3.30,3.12,3.15,219600,0,0.0
1980-03-18,0.0,3.12,2.94,3.03,727200,0,0.0
1980-03-19,0.0,3.08,3.02,3.04,295200,0,0.0
1980-03-20,0.0,3.06,3.01,3.01,159600,0,0.0
1980-03-21,0.0,3.02,2.91,2.92,130800,0,0.0


In [10]:
from googlefinance import getQuotes
import json

In [12]:
json.dumps(getQuotes('AAPL'), indent=2)

HTTPError: HTTP Error 403: Forbidden